# Database SchemaModel Serialization

This is the the sandbox for extending the database SchemaModel towards 

- Generating the base SchemaModel from a sql query 
  - [ref](../Sql/Discovery-pk-fk.sql)
- easily inserting 
  - implied links
  - Parameters/External links
- deciding bast defaults for Sorters and Filters 
  - and do these need to be bytes rather than bools?
- Devise a good generic SOLID solution for Special Handling

In [ ]:
// import packages and live code classes
// packages:
#r "nuget: Microsoft.Data.SqlClient"


In [ ]:
// import composite containing all support classes ...

#!import "../code/composite.cs"

## To Demo that the library is working

open (or move to where you can see it) a browser.

In [ ]:
// Quick demo to test library:
// Load some existing Json:
var json = FileData.FromFile("../.data/instances-raw.json");
// Generate a page from that:
var page = Nest.RenderPageStepping(json,10,"Hello Json-to-Html!");
// Push to a file and load into the browser:
var htmlPath = "../.html/Schema-page.html";
_ = FileData.ToFile(page, htmlPath);
FileData.RunFile(htmlPath);

## Context for the SchemaModel

The starting point for building a SchemaModel is to extract from the existing database (or perhaps a single schema)
- The schema.table.column info
- Primary Keys
- Foreign Key

We will extract this info as json using a sql query and deserialize into the base SchemaModel.

(We want the SchemaModel to be json serializable for ease of storage and exchange)





In [ ]:
// add sql kernel
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"


In [ ]:
#!powershell
docker start awsql


In [ ]:

// import a connection string


#!import ..\.data\connect-aw-db-dox.csx

In [ ]:
#!sql-aw-db --name pkReferences
--- 
select kcu2.table_schema + '.' + kcu2.table_name + '.' + kcu2.column_name as PrimaryKey
, kcu1.table_schema + '.' + kcu1.table_name + '.' + kcu1.column_name as ForeignKey
from information_schema.referential_constraints rc
join information_schema.key_column_usage kcu1
on kcu1.constraint_catalog = rc.constraint_catalog 
   and kcu1.constraint_schema = rc.constraint_schema
   and kcu1.constraint_name = rc.constraint_name
join information_schema.key_column_usage kcu2
on kcu2.constraint_catalog = rc.unique_constraint_catalog 
   and kcu2.constraint_schema = rc.unique_constraint_schema
   and kcu2.constraint_name = rc.unique_constraint_name
   and kcu2.ordinal_position = kcu1.ordinal_position

In [ ]:
// pass the sql results to C#

#!share --from sql-aw-db pkReferences

pkReferences.Display();

## So ...

We have set up a system whereby we can interact with the sql and c# environments happily.
this allows us to develop scripts and code in parallel in the same window.

Some notes on the above:

- Import packages in a cell that only imports libraries, otherwise the other code will try to run before the dependency has finished loading.
- When it comes to importing code directly, I found that code which depends on other classes should be imported in the same file or you will get strange errors, even if the previous file contained the dependencies.
- It is quite feasible to run the sql connection string by importing it, which means we don't have to make the credentials visible in this file
- This environment can be Gitted, as long as the folders with a leading period are excluded in the gitignore file: this allows us to keep stuff private.
- When rerunning sql, the kernel will tell you it is already connected: avoid this by only rerunning the latest queries
- The #!sql... command should be the first line in the cell!
- It is easy to pass the results to C# for serialization or whatever, as demonstrated above.

In [ ]:
#!sql-aw-db --name allColumns
--select * from information_schema.columns
select table_schema + '.' + table_name + '.' + column_name as 'Identity',
CASE IS_NULLABLE WHEN 'yes' Then 'true' ELSE 'false' END as IsNullable,
DATA_TYPE as FieldDatatype
from information_schema.columns
select distinct data_type from information_schema.columns

In [ ]:
// This needs to go into the Fields class for rationalization

enum FieldDataTypes 
{
    Bit,
    Date,
    Datetime,
    Decimal,
    Geography,
    Hierarchyid,
    Int,
    Money,
    Nchar,
    Numeric,
    Nvarchar,
    Smallint,
    Smallmoney,
    Time,
    Tinyint,
    Uniqueidentifier,
    Varbinary,
    Varchar,
    Xml
}

In [ ]:
#!sql-aw-db --name fieldDefinitions

with c as (
select 
table_schema SchemaName,
table_name TableName,
column_name FieldName,
lower(table_schema + '.' + table_name + '.' + column_name) as 'identifier',
CASE is_nullable WHEN 'yes' Then 'true' ELSE 'false' END as IsNullable,
DATA_TYPE as FieldDatatype
from information_schema.columns
),
k as
 (
select 
 lower(kcu2.table_schema + '.' + kcu2.table_name + '.' + kcu2.column_name) as Identifier
, lower(kcu1.table_schema + '.' + kcu1.table_name + '.' + kcu1.column_name) as Reference
from information_schema.referential_constraints rc
join information_schema.key_column_usage kcu1
on kcu1.constraint_catalog = rc.constraint_catalog 
   and kcu1.constraint_schema = rc.constraint_schema
   and kcu1.constraint_name = rc.constraint_name
join information_schema.key_column_usage kcu2
on kcu2.constraint_catalog = rc.unique_constraint_catalog 
   and kcu2.constraint_schema = rc.unique_constraint_schema
   and kcu2.constraint_name = rc.unique_constraint_name
   and kcu2.ordinal_position = kcu1.ordinal_position
) 
select top 5 
c.Identifier, FieldName,TableName,SchemaName,IsNullable, FieldDatatype, Reference
 from c left join k on c.[identifier] = k.identifier
 -- where reference is not null
 for json path, include_null_values


In [ ]:
#!share --from sql-aw-db fieldDefinitions

var k = fieldDefinitions;
// The result is a array to accommodate potential multiple rowsets. 
// We could access the Data and Schema properties but extracting the json works fine!
FileData.ToFile(k[0].ToJsonString().ToString(), "../.data/fieldDefinitions00.json");


Both the notebook and the SSMS interface do a bunch of strange stuff with the results of a json query. We have already managed this awkwardness in our library, so let's use that instead.



In [ ]:
var pw = Environment.GetEnvironmentVariable("AW_DB_DOX");
var cs = $"Server=127.0.0.1,2433; User=sa; Password={pw}; Initial Catalog=AdventureWorks2022;trust server certificate=true";
var _sql = new JsonSqlClient(cs);

var query = 
  "with Data as ("
+ "select  "
+ "table_schema SchemaName, "
+ "table_name TableName, "
+ "column_name FieldName, "
+ "lower(table_schema + '.' + table_name + '.' + column_name) as 'identifier', "
+ "CASE is_nullable WHEN 'yes' Then 'true' ELSE 'false' END as IsNullable, "
+ "DATA_TYPE as FieldDataType "
+ "from information_schema.columns "
+ "), "
+ "k as "
+ " ( "
+ "select  "
+ " lower(kcu2.table_schema + '.' + kcu2.table_name + '.' + kcu2.column_name) as Identifier "
+ ", lower(kcu1.table_schema + '.' + kcu1.table_name + '.' + kcu1.column_name) as Reference "
+ "from information_schema.referential_constraints rc "
+ "join information_schema.key_column_usage kcu1 "
+ "on kcu1.constraint_catalog = rc.constraint_catalog  "
+ "   and kcu1.constraint_schema = rc.constraint_schema "
+ "   and kcu1.constraint_name = rc.constraint_name "
+ "join information_schema.key_column_usage kcu2 "
+ "on kcu2.constraint_catalog = rc.unique_constraint_catalog  "
+ "   and kcu2.constraint_schema = rc.unique_constraint_schema "
+ "   and kcu2.constraint_name = rc.unique_constraint_name "
+ "   and kcu2.ordinal_position = kcu1.ordinal_position "
+ ")  "
+ "select  "
+ "data.Identifier, FieldName,TableName,SchemaName,IsNullable, FieldDataType, Reference "
+ " from data left join k on data.[identifier] = k.identifier "
+ " -- where reference is not null "
+ " for json path, include_null_values ";

In [ ]:
var x = _sql.JsonSingleUsingQuery(query);

In [ ]:
public class FieldInfo
{
    public string Identifier { get; set; }
    public string FieldName { get; set; }
    public string TableName { get; set; }
    public string SchemaName { get; set; }
    public string IsNullable   { get; set; }
    public string IsExcluded {get; set; }
    public string IsEnabled { get; set; }
    public string FieldDataType { get; set; }
    public string Reference { get; set; }
    public override string ToString()
      => $"{SchemaName}.{TableName}.{FieldName}<{FieldDataType}>";
    public string PreProcess { get; set; }
    public string PostProcess { get; set; }
}

In [ ]:
var y = JsonSerializer.Deserialize<FieldInfo[]>(x);

y.Count().Display();
for (int ix = 50; ix < 54; ix++)
y[ix].Display();
foreach ( var f in y.Where(y => y.Reference.Length > 12))
   f.Identifier.Display();
foreach ( var f in y.Where(y => y.SchemaName == "Sales"))
   f.TableName.Display();


This allows us to build the SchemaModel directly from the database query. This can be build into the SchemaModeling class.
For efficiency, we could easily wrap the array in a dictionary (or multiple dictionaries) for speedy service: these would be generate from Linq queries of the SchemaModel vey easily.

Process for building a SchemaModel

! - GenerateModel(database)
    This makes the full field list and imports all key references.

    Exclude(*.*.*) schema, table or field wildcards
    Include(*.*.*)

    AddImpliedReference
    AddParameter

If something is excluded, any references to it will generate a variable.
If it is not enabled, it is effectively ignored completely

Once we have done this, we have the operational data SchemaModel and have defined the scope of our queries.

Some fields have composite content.
Where this is the case, we might have to preprocess the data, for example
- decompress
- decrypt
- redact
- look up


The data might also result in additional information structure, for example a json-encapsulated value could expand into a subtree. At this point the presentation schema departs from the storage schema

PostProcessing would occur as the last step, typically to format, truncate or simplify the data.



# Process of ingesting a schema into this SchemaModel.

Using the database, we initialize a SchemaModel in a single pass, which gets all field definitions across all schemas. PK-FK relationships are detected.

(The SchemaModel is from this point on serializable as json. Because the fields are stored in an array, they can be iterated through using Linq.)

### Schema and Scope

Schemas never intended to be used can be stripped before the SchemaModel is saved. 

As an alternative to removing unwanted schemas, we an simply add exclusions to get focused on our main area of interest.

### References

References which are excluded (or not found) automatically become available as external parameters.

We can also add implied references and parameters. We will use this SchemaModel to generate design-time queries and sub-models to use at run-time.



### Query Generation

The current query generator simply takes the Links provided and generates Sql (by deciding whether the links are peers or nestings) and returns the query, along with a text output describing the way the links were resolved (for diagnostic purposes during development  mainly).
In our case we have way more information than just the links already.

The current process for generating the sql involves 
- an arbitrary list of links is passed into the GenerateNestedQuery method.
- this calls into the ProcessMutliLinkQuery method with recursion.
  - The series is started with an empty output array and a list of links.
    - initially:
      - the first link is made current
      - it is added as root
      - t is removed from the list
      - we recurse.
    - therafter
      - Each successive link is evaluated and we determine whether it should be added as a peer or added as a nest.
      - we either add as nest or add as a peer.
    - The Add As Root, Add as  Nest and Add As Peer calls are themselves recursive:
      - They are very similar in that they dynamically get the column names.
      - ***It is feasible to use the GetColumnNames call to question the SchemaModel and inject any controls there!***

Example select statements:
Select 
    Name, City from Addresses
    `'<!pre!redact!>' + Name` as Name, City
    `'<!pre!decrypt!!post!hash!>' + Name` as name, City

In this way the data returned from the database can be tagged for special handling by the postprocessor later (when it prepared for rendering).



**CHALLENGE**

When the preprocess results in an expansion, we cannot do that until we have the data resulting from the query. When we do that, we will need to intercept the value when the nest ios being made, and process accordingly.
this needs to be looked at in the Nest routine that generates the html.

This occurs in `AsHtmlStepping` which builds a start and end html string, returning the two together.
This is totally recursive, and has different handlers based on the type of Nesting
- Item
- list
- SetList
- Set

A strategy for dealing with this might be
- The Pre and Post clearly live in the Values.
- If Nest contains a pre/post, this can be detected at the moment the nest enters the AsHtmlStepping method. the handler can process the nest, and return the nest in its modified form!
- it then passes into the Nesting switch and we are DONE.
- the Processors are really the same whether post or pre!A processor could be defined with 
  - the activating Token
  - a chain of lambdas for the process chain
- The Nest would call the Lambda processor and return the modified Nest.
- the recursion should take care of the children just fine.

**Implies: we insert the detection at the start of the html process before the Nesting branch and pass to a handler to process based in the token and the content!**

We can define the query scope by explicitly choosing the links to traverse, 
or define a start and end table and have the system generate the possible paths from the known links. 

Perhaps as an interim, if we have a link follower, we cold get a list of potential tables, from which we could exclude.

Either way, we have a query that will harvest the data, inject the appropriate column data for pre and post processing.

When we run the query, it generates json, which is storable.

The schema and data are in the form of Json.



## Further thoughts:

What we need to achieve:
- Retrieve meaningful data including relationships
- Decode, extract and redact some of that data to allow it to be used more freely
- The ability to save, compare and view this data offline  

How we provision this:
- At Design-time, we SchemaModel the data schema
  - Manual steps are required only for defining special behavior, the rest is reflected.
- At run-time
  - Parameters (or limits) are provided
  - The system builds and executes the sql
  - Results are returned as json
  - The system post-processes the data for expansion, redaction, whatever is required.
  - The output of this is Html, which can be viewed, stored, shared and reviewed offline.

Assuming a process where 

- ** At Design Time**
  - a SchemaModel has been set up for a particular function 
  - or a previously saved SchemaModel is loaded
  - available parameters/filters are defined

- ** At run time
  - The sql is generated using the SchemaModel and any supplied parameters/filters
  - The generated sql aliases the column names with a key containing the field indexes. 
  - The values are received back from the query as json serialized objects.

  When we process the data that comes back:
  - We use the index to retrieve the field metadata
  - Pre- and post- processing can be applied where indicated by the field metadata data
  - We intercept right at the point where the html generator is about to determine the structure
  - This allows us to inject or remove structured data












## The main Generator process

```mermaid
flowchart LR
subgraph Context
u(((user))) 
u --> g0
db[(Database)]
js(JsonSql)
fb{{FileBin>}}
subgraph Generator
g0[[Get Context]]
g0 --> g1[[Retrieve SchemaModel]]
g1 -->g2[[Inject Filters]]
g2 --> g3[[Generate Query]]
g3 --> g4[[Submit Query]]
jr[/JsonResults/]
jr --> g5[[PostProcess results into Nest]]
g5 --> g6[[Persist Nest]]
g6 --> g7[[Convert Nest to Html]]
m[/SchemaModel/]
end
g1 --> m
m --> g3
m --> g5
g1 <-.-> fb
g6 <-.-> fb
g4 --> js
js --> db
db --> js
js --> jr
g7 --> u
end
```

## As of now ...

The process seems sound, but the libraries might need some refactoring and renaming.

Responsibilities:

- JsonSql 
  - allows JsonSqlClient
  - gets sql to the db and returns json.
- FileData (migrating to FileBin...?) 
  - supports various simple file operations
- Nest
  - Manages hierarchial data
  - Generates Css, Js and Html, and full pages from json data
  - Generates Sql from a list of links

The Nest seems to be a little confused, it should probably break away

- Nest
  - Manage hierachical Data
  - Generate Nest from json data
- QueryGenerator
  - Generate sql queries from a ~~nest~~ SchemaModel
  - (could do other queries in the same way!)
- Html Generator
  - Generate Html frp, a nest
At the moment the Nest does too much because it is responsible fopr both generating the prequery and converting the post query data.
It was also doing too little because we need to Postprocess the data before convertying to html.



## SchemaModel-generator - Sql-generator - Json-translator - View-Generator

Splitting the process into atomic operations (<u>S</u>-olid)

Design-time:
- SchemaModel Generation queries the database to produce a base SchemaModel
- SchemaModel Refinement modifies the SchemaModel for a specific purpose

Run-time: 
- Sql Generator combines a SchemaModel with run-time filters to produce a sql query
- A JsonSql handler executes the query and returns SchemaModel-indexed json
- A PostProcessor uses the SchemaModel to reconstitute the json data
- A View Generator converts the json to a viewable html page or panel










## Classes for minimal iterability

```Mermaid
classDiagram
class SchemaModeler{
    Generate(database) 
    Refine(qualifications)
    Save()
    Load()
    PostProcess(SchemaModel) : json
}
class QueryGenerator{
    GenerateQuery(model, filters) : query
}
class JsonSql{
    JsonFromSingleQuery(query) : json
}
class PageProducer{
    MakePage(json, options) : html
}
class FileBin {
    Save()
    Load()
    Launch()
}
```

## Clearer version 

The SchemaModeler produces (and modifies) a SchemaModel representing the Database Schema.
The QueryGenerator uses a (possibly modified) SchemaModel and optional filters to generate and execute a query to retrieve data using the SchemaModel, in Json format.
The NestedDataModeler uses the SchemaModel to post-process the raw results, converting these to a serializable hierarchical data structure (Nested Data) including both information and the hierarchical data structure. 
The NestedDataPresenter accepts the NestedData and converts this to a displayable form using html, css and javascript.
```mermaid
classDiagram 
Direction LR


ISchemaModeler <|-- SqlSchemaModeler
SqlSchemaModeler  *-- SqlJsonClient
SqlJsonClient *-- SqlDataSource
SqlSchemaModeler *-- SchemaModel
SchemaModel *-- BaseFields
SchemaModel *-- Overlays
SchemaModel ..|> SchemaModelFile
ModelFile ..|> SchemaModel

IModelQuery <|-- SqlModelQuery
SqlModelQuery *-- Filters
Filters ..> SchemaModel
SqlModelQuery *-- SchemaModel
SqlModelQuery --|> QueryResult
SqlModelQuery ..> SqlJsonClient

SqlModelQuery ..|> NestedDataModeler

INestedDataModeler <|-- NestedDataModeler
NestedDataModeler *-- SchemaModel
NestedDataModeler *-- QueryResult
NestedDataModeler ..|> NestedData
NestedData ..|> NestedDataFile
NestedDataFile ..|> NestedData

NestedDataPresenter <|-- NestedDataPresenter
NestedDataPresenter *-- NestedData
NestedDataPresenter ..|> Page

class ISchemaModeler {
    <<interface>>
    ctor(jsonSql)
    Initialize()
    Refresh()
    Overlay()
    Save()
    Load()
    Prune()
}
class IModelQuery{
    <<interface>>
    ctor(jsonsql, SchemaModel)
    GetFilters()
    Execute(filters)
    ExecuteData(path, filters)
    ExecutePage(path, filters)
}
class INestedDataModeler{
    <<interface>>
    SchemaModel()
}
class NestedDataPresenter{
    <<interface>>
    +PresentationStyle
    +Options
    GetPage(options)

}
class SchemaModelFile{
    <<json>>
}
class QueryResult{
    <<json>>
}
class NestedData{
    <<json>>
}
class NestedDataFile{
    <<json>>
}
class Page{
    <<html+css>>
}


## Sequence

```mermaid
sequenceDiagram
Actor d as Designer
participant dm as ISchemaModeler
participant mq as IModelQuery
participant rm as INestedDataModeler
participant pp as NestedDataPresenter
participant fs as File System
participant mm as SchemaModel
participant js as JsonSqlClient
participant QH as Query Handler
Actor u as User

d ->> dm : Initialize()
dm -->> mm : Base SchemaModel
alt
d ->> dm : Save()
dm -->> fs : <model>
end
alt
d ->> dm : Load()
fs -->> dm : <model>
end
d ->> dm : Overlay()
u ->> QH : Execute(Query, filters)
QH ->> mq : Init(query)
QH ->> mq : Execute(filters)
mq ->> mm : Get
mq ->> js : <query>
mq ->> rm : SchemaModel(queryResult)
alt
rm -->> fs : <nested data>
end
rm ->> pp: GetPage()
pp -->> rm : <page>
rm -->> u : <page>



## Designer Initializes a Data SchemaModel
The connection string to the database is injected into a JsonSqlClient.
The JsonSqlClient is then in injected into an IDataModeler instance.
Then,
- Initialize() can generate the base SchemaModel
- Overlay() can customize the SchemaModel
- Refresh() can update the base SchemaModel under the overlays
- Save() and Load() can persist the SchemaModel
- Prune() can make a pruned version of the SchemaModel.

## Executes a query 

To execute a query (for later publication), 
- The IModelQuery must be instantiated with the JsonSql source and the applicable SchemaModel
- The Execute function can be called with or without filters.
- The resulting QueryResult is passed into an instance of the NestedDataModeler
- The result is storable NestedData
- It would be possible to inject a NestedDataPresenter into the constructor too and generate the page immediately.



## Publish Nested Data

To publish nested data, 
- Instantiate an NestedDataPresenter with appropriate options 
- Either call GetPage, or assemble your page using the other calls.

## UI Connection issues

There may be a need for some kind of Query Manager to
- Locate the correct SchemaModel
- Connect the correct DataSource to the DataModeller
- Offer the correct Filters to the UI
- Optionally configure a NestedDataPresenter
- Pass the SchemaModel and Data Client and optionally the NestedDataPresenter to the SchemaModelQuery
- Persist the Nested Data and/or the Page or both

Generally, there might be a need to configure how and when various intermediary products are persisted, temporarily of permanently.

Always persist or stream:
- SchemaModels
- Pages
Likely persist for reuse in a diagnostic situation:
- NestedData
Preferably throw away:
- QueryResult

There are not many variants of this, perhaps we could control this with a single PersistanceFlags enum.

Also, viewing this as a Data Pipeline, it is an excellent candidate for Flow Management.

## New thoughts (random)

- The field defs could have a Class which could be transmitted to the UI! Or we could generate on e based on a property.
- The fields could have an importance which could translate to a class on the FE and also be used to generate a prioritized potential-for-filter candidate list.
- There is not much overhead in saving the index/field-code in the field...probably less than using a dictionary.
- It might be useful to auto-limit the initial SchemaModel build by schema(s)
- Using the concept of a base SchemaModel and overlays, for this to idempotent (so that the base can be refreshed) we need to use an open-closed principle in that the base data should not be changed by the overlays directly.
  - This means that everything potentially applied may modify the interpreted result but never affects the base properties, 
    - identity
    - data type
    - schema/table/column
  - but allows run-time variants to persist
    - disabling
    - exclusion
    - internal and external parameters



SOLID revisited

- Single responsibility: Not only that, but we need to embrace the idea that an operation comes into existence for a single atomic purpose, awakes some state and finally deposits the results. This should be sufficient for future operations to act appropriately.
- Open-Closed: When we are building state from sequential operations, we should consider this to be a base operation with overlaid operations, where the original can be repeated without corrupting the overlaid state, to allow out-of-scope evolution to be incorporated.
- Liskov: The interfaces required for the structural organization of effort should be as simple as possible to allow switching providers with minimal complexity and no alteration of flow
- Interfaces: These should be structural in nature, defining the way things are assembled and flow. The generic portions are allow diverse operations to mesh cleanly.
- Dependencies: We need dependencies to bubble up to a layer where they can be easily inherited or injected at runtime. This is also critical for data privacy and security.




## Components

### JsonSqlClient
```mermaid
flowchart LR
cs[/Connection String/] -.-> jc[[JsonSqlClient]]
q[[JsonSingleUsingQuery]] --> jc
jc -.-> j(json)
db[(Database)] -.-> jc
```



### SchemaModeler

```mermaid
flowchart LR
subgraph input
cs[/Connection String/] 
end
cs -.-> dm[[SchemaModeler]]
i[[Initialize]] --> dm
r[[Refresh]] --> dm
p[[Prune]] --> dm
o[[Overlay]] --data--> dm
s[[Save]] --path--> dm
l[[Load]] --path--> dm
dm <-.-> f
subgraph output
f(SchemaModel File)
end
```


### SchemaModelQuery + NestedDataModeler

```mermaid
flowchart LR
subgraph input
cs[/Connection String/] 
m[/SchemaModel/] 
end
m -.-> mq
cs -.-> mq[[SchemaModelQuery]] 
mq -.-> mr[/ModelResults/] 
mr -.-> rm[[NestedDataModeler]]
m -.-> rm
gf -.-> m
x[[Execute]] --filters--> mq

gf[[GetFilters]] --> mq
rm -.-> nd
subgraph output
nd(NestedData)
end
```

### NestedDataPresenter

```mermaid
flowchart LR
subgraph input
o[/Options/] 
nd(NestedData)
end
nd -.-> p[[NestedDataPresenter]]
o -.-> p[[NestedDataPresenter]]
pp[[GetPage]] --> p
subgraph output
h(html+css)
end
p -.-> h


## Dependencies

```mermaid
flowchart TD
dm[DataModeler]
js{{JsonSql}}
mq[ModelQuery] 
rm[NestedDataModeler]
pp[NestedDataPresenter]
nd{{NestedData}}
fd{{FileData}}
mm{{Model}}

fm{{Coordinator}}
mq -.->|T| fm
rm -.->|T| fm
pp -.->|T| fm

dm -.-> js
dm -.-> mm
mq -.-> js
mq -.-> mm
rm -.-> nd
rm -.-> mm
pp -.-> nd
dm -.-> fd 
mq -.-> fd
rm -.-> fd
pp -.-> fd

```





## Coordinator

The tasks managed by the main classes can be simplified by a dedicated coordinator.
To allow this, each must support a generic function that can be used to chain activity one to another. This is possible because the input and output types are predictable.

An outside class can call the coordinator with instances of the classes, and any parameters required in addition to those cascaded

e.g. `Coordinate([myModelQuery, myNestedDataModeler, MyNestedDataPresenter], connectionString, SchemaModel, PublicationOptions)` which will result in a call to

`myModelQuery(connection, SchemaModel)` passing its output to 
`myNestedDataModeler()` which would pass its result to 
`myNestedDataPresenter(options)` which would ultimately return its result to the calling class.

The parameters array would be consumed by the Process call as required by the participants.
The whole coordinated process should occur asynchronously and should return a CoordinatedResult which can allow for and encapsulate errors.
Passing a CoordinatedInput of T allows the receiver to access and consume parameters;
Returning a CoordinatedOutput of T allows management of flow and errors and possibly cancellation or status

Why do this?
Because it limits interdependency between chained operations making them simpler and more single-minded.

## ICoordinator - ICoordinated

```mermaid
classDiagram
class ICoordinated{
    <<interface>>
    Process (CoordinatedInput<T>)  CoordinatedOutput<T>
}
class ICoordinator{
<<interface>>
    Coordinate(ICoordinated[], param [])

}

```

Any Coordinator-derived class can provide entry points to chain any kind of coordinated activities. Because the type can be defined to match the provider's requirements, the actual coordinator class is independent of its users, although the derived class can use (or overide) the abstract base class implementation. The coordinator is of course a thinly-veiled minimal implementation of a Flow Manager capable of pessimistic flow control.